### DSC vs. NSD, as appropriate, by organ

#### Most basic - needs csv

In [7]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df_metric = pd.read_csv("organ_perf_dsc_merged.csv")

# Display the first few rows
print(df_metric.head())


    case  organ     value        algo
0  s0001  aorta  0.896496  _Diff-UNet
1  s0006  aorta  0.968756  _Diff-UNet
2  s0009  aorta  0.982722  _Diff-UNet
3  s0012  aorta  0.889999  _Diff-UNet
4  s0013  aorta  0.979722  _Diff-UNet


In [8]:
unique_organs = df_metric["organ"].unique()

for org in unique_organs:
    # Subset just this organ
    sub_df = df_metric[df_metric["organ"] == org]

    # Group by algo, compute mean of value
    agg_df = (
        sub_df
        .groupby("algo", as_index=False)["value"]
        .mean()
        .rename(columns={"value": "mean_value"})
    )

    # Sort descending if bigger metric => better
    agg_df = agg_df.sort_values("mean_value", ascending=False)

    # Assign a rank (1 = best)
    agg_df["rank"] = range(1, len(agg_df) + 1)

    print("=== Organ:", org, "===")
    print(agg_df.to_string(index=False))
    print()


=== Organ: aorta ===
                           algo  mean_value  rank
              nnU-Net_STU-Net_L    0.740417     1
              nnU-Net_STU-Net_H    0.726646     2
              nnU-Net_STU-Net_B    0.718101     3
                nnU-Net_ResEncL    0.713468     4
                nnU-Net_MedNeXt    0.695329     5
                  nnU-Net_U-Net    0.684460     6
                     _MedFormer    0.678518     7
                 nnU-Net_UniSeg    0.673310     8
                 nnU-Net_NexToU    0.595173     9
                  MONAI_LHU-Net    0.591955    10
                     _Diff-UNet    0.577087    11
                   MONAI_SegVol    0.576092    12
     Vision_Language_U-Net_CLIP    0.574811    13
Vision_Language_Swin_UNETR_CLIP    0.565573    14
                    MONAI_UNEST    0.515975    15
               MONAI_Swin_UNETR    0.511609    16
               MONAI_UCTransNet    0.481466    17
                    MONAI_UNETR    0.368090    18
                   _SAM-Adapt

### Reimplementation of R code in Py

In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import rankdata
from scipy.spatial.distance import pdist, squareform

# Path to parent folder containing one subfolder per algorithm
parent_dir = "./totalsegmentator_results/final_14dec2024_results/totalsegmentator_results - table 3 in paper"

# List subdirectories (one per algorithm)
model_dirs = [os.path.join(parent_dir, d) for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

# Our 9 organs of interest
all_organs = ["aorta", "gall_bladder", "kidney_left", "kidney_right",
              "liver", "pancreas", "postcava", "spleen", "stomach"]

# Decide which metric to use for each organ: DSC vs NSD
organs_use_nsd = {"aorta", "postcava"}
organs_use_dsc = set(all_organs) - organs_use_nsd

# Directory to save plots (optional)
out_plots_dir = "organ_plots"
os.makedirs(out_plots_dir, exist_ok=True)


def read_metric_data(metric, organs):
    """
    Read the CSV data from each algorithm folder and pivot to long format.
    
    Parameters:
        metric (str): "dsc" or "nsd"
        organs (list): List of organs to extract

    Returns:
        pd.DataFrame: Combined long-format data from all algorithm results
    """
    all_data = []

    for mdir in model_dirs:
        csv_file = os.path.join(mdir, f"{metric}.csv")
        if not os.path.exists(csv_file):
            print(f"Skipping: no {metric}.csv in {mdir}")
            continue

        df = pd.read_csv(csv_file)

        if 'name' not in df.columns:
            print(f"Skipping: {csv_file} missing 'name' column")
            continue

        df_long = df.melt(id_vars=['name'], value_vars=organs, var_name='organ', value_name='value')
        df_long.rename(columns={'name': 'case'}, inplace=True)
        df_long['algo'] = os.path.basename(mdir)

        # Remove rows with NaN values
        df_long = df_long.dropna()

        all_data.append(df_long)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame(columns=['case', 'organ', 'value', 'algo'])


def compute_ranking(df):
    """
    Compute ranking of algorithms using statistical significance.

    Parameters:
        df (pd.DataFrame): Dataframe containing algorithm performance

    Returns:
        pd.DataFrame: Dataframe with ranking per algorithm
    """
    algo_means = df.groupby('algo')['value'].mean()
    rankings = rankdata(-algo_means, method='min')  # Higher value = better rank
    rank_df = pd.DataFrame({'algo': algo_means.index, 'mean_score': algo_means.values, 'rank': rankings})
    return rank_df.sort_values(by='rank')


def bootstrap_ranking(df, nboot=200):
    """
    Perform bootstrapping to compute rank distributions.

    Parameters:
        df (pd.DataFrame): Algorithm performance data
        nboot (int): Number of bootstrap samples

    Returns:
        pd.DataFrame: Bootstrapped rank distributions
    """
    boot_ranks = []
    algos = df['algo'].unique()
    
    for _ in range(nboot):
        sample = df.sample(frac=1, replace=True)
        rank_sample = compute_ranking(sample)
        boot_ranks.append(rank_sample[['algo', 'rank']])

    boot_df = pd.concat(boot_ranks)
    return boot_df


# Store results (tables/plots) in a dictionary
results_per_organ = {}

for org in all_organs:
    # Decide which metric to use
    metric_choice = "nsd" if org in organs_use_nsd else "dsc"

    # Read data for the organ
    df_metric = read_metric_data(metric_choice, [org])

    if df_metric.empty:
        print(f"No data found for organ {org} with metric={metric_choice} -- skipping.")
        continue

    # Compute ranking
    rank_df = compute_ranking(df_metric)

    # Perform bootstrapping for violin plot
    boot_df = bootstrap_ranking(df_metric, nboot=200)

    # Generate Figures

    ## Boxplot
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df_metric, x='algo', y='value', showfliers=False)
    sns.stripplot(data=df_metric, x='algo', y='value', jitter=True, alpha=0.6, color='black')
    plt.xticks(rotation=90)
    plt.title(f"Boxplot for {org} using {metric_choice.upper()}")
    plt.xlabel("Algorithm")
    plt.ylabel(metric_choice.upper())
    plt.savefig(os.path.join(out_plots_dir, f"box_{org}_{metric_choice}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    ## Significance Map (Fixed)
    algos = rank_df['algo'].values
    ranks = rank_df['rank'].values.reshape(-1, 1)
    rank_matrix = squareform(pdist(ranks, metric='euclidean'))  # Pairwise ranking distances

    sig_df = pd.DataFrame(rank_matrix, index=algos, columns=algos)

    plt.figure(figsize=(8, 6))
    sns.heatmap(sig_df, annot=True, cmap="coolwarm", cbar=True)
    plt.title(f"Significance Map for {org} ({metric_choice.upper()})")
    plt.savefig(os.path.join(out_plots_dir, f"sigmap_{org}_{metric_choice}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    ## Violin Plot of Bootstrapped Ranks
    plt.figure(figsize=(8, 6))
    sns.violinplot(data=boot_df, x='rank', y='algo', inner="quartile")
    plt.title(f"Violin of Ranks: {org} ({metric_choice})")
    plt.savefig(os.path.join(out_plots_dir, f"violin_{org}_{metric_choice}.png"), dpi=300, bbox_inches='tight')
    plt.close()

    # Store results
    results_per_organ[org] = {
        "organ": org,
        "metric": metric_choice,
        "ranking_df": rank_df,
        "bootstrap_ranks": boot_df
    }

print("Processing completed. Results saved.")

Processing completed. Results saved.
